In [1]:
import pandas as pd
data = pd.read_csv('Data/train.csv')
data.sample(2)

,Image,target
289,207.jpg,kathakali
337,167.jpg,bharatanatyam


In [2]:

from sklearn.model_selection import train_test_split
train, val = train_test_split(data, test_size = 0.13, random_state = 19)

In [3]:
import os
os.mkdir('Data/final_train')
os.mkdir('Data/final_test')
os.mkdir('Data/final_val')

dance_forms = data['target'].unique().tolist()
for i in dance_forms:
  os.mkdir('Data/final_train/'+i)
  os.mkdir('Data/final_val/'+i)

In [4]:
import shutil

s = 'Data/train/'
d1 = 'Data/final_train/'
d2 = 'Data/final_val/'

train_im = train['Image'].array
train_t = train['target'].array
for i in range(0, len(train_im)):
  im = train_im[i]
  target = train_t[i]
  shutil.copy(s + im, d1 + target + '/' + im)

val_im = val['Image'].array
val_t = val['target'].array
for i in range(0, len(val)):
  im = val_im[i]
  target = val_t[i]
  shutil.copy(s + im, d2 + target + '/' + im)

test = pd.read_csv('Data/test.csv')
for im in test['Image'].array:
  shutil.copy('Data/test/' + im, 'Data/final_test/' + im)

In [5]:

import os
import cv2
from PIL import Image

In [6]:

def resize(s,src,des):
  size = s,s
  for fname in os.listdir(src):
    spath = os.path.join(src+fname)
    dpath = os.path.join(des+fname)
    if os.path.isfile(spath):
      image = Image.open(spath)
      image.thumbnail(size, Image.ANTIALIAS)
      image.save(dpath)

def add_padding(s,src,des):
  for fname in os.listdir(src):
    spath = os.path.join(src+fname)
    dpath = os.path.join(des+fname)
    if os.path.isfile(spath):
      im = Image.open(spath)
      desired_size = s
      old_size = im.size
      ratio = float(desired_size)/max(old_size)
      new_size = tuple([int(x*ratio) for x in old_size])
      im = im.resize(new_size, Image.ANTIALIAS)
      new_im = Image.new("RGB", (desired_size, desired_size))
      new_im.paste(im, ((desired_size-new_size[0])//2,(desired_size-new_size[1])//2))
      new_im.save(dpath)

In [7]:
dance_forms = [i+'/' for i in dance_forms]

for i in ['Data/final_train/','Data/final_val/']:
  for j in dance_forms:
    resize(512, i + j, i + j)
    add_padding(512, i + j, i + j)

k = 'Data/final_test/'
resize(512, k, k)
add_padding(512, k, k)

In [ ]:
import random
from scipy import ndarray
import skimage as sk
from skimage import transform, util, io

# Random rotation within specified upper and lower degree bounds
def random_rotation(image_array: ndarray, lb, ub):
  random_degree = random.uniform(lb, ub)
  return sk.transform.rotate(image_array, random_degree)

# Random noise into the image
def random_noise(image_array: ndarray):
  return sk.util.random_noise(image_array)

# Horizontal Flip
def horizontal_flip(image_array: ndarray):
  return image_array[:, ::-1]

# Bluring the image
def blur_image(image):
  return cv2.GaussianBlur(image, (9,9),0)

# Shift pixels by an offset
def warp_shift(image): 
  transform = sk.transform.AffineTransform(translation=(10, 10))
  warp_image = sk.transform.warp(image, transform, mode="wrap")
  return warp_image

# Gray image
def gray(image):
  return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [ ]:
for i in dance_forms:
  aug = 0
  folder_path = os.path.join('Data/final_train/',i+'/')
  for fname in os.listdir(folder_path):
    if os.path.isfile(folder_path + fname):
      image_to_transform = sk.io.imread(folder_path + fname)
      #1
      transformed_image = horizontal_flip(image_to_transform)
      transformed_image = blur_image(transformed_image)
      aug += 1
      io.imsave(folder_path + 'aug' + str(aug) + '.jpg', transformed_image)
      #2
      transformed_image = random_rotation(image_to_transform, -45, 0)
      transformed_image = random_noise(transformed_image)
      aug += 1
      io.imsave(folder_path + 'aug' + str(aug) + '.jpg', transformed_image)
      #3
      transformed_image = horizontal_flip(transformed_image)
      transformed_image = blur_image(transformed_image)
      aug += 1
      io.imsave(folder_path + 'aug' + str(aug) + '.jpg', transformed_image)
      #4
      transformed_image = random_rotation(image_to_transform, 0 ,45)
      transformed_image = random_noise(transformed_image)
      aug += 1
      io.imsave(folder_path + 'aug' + str(aug) + '.jpg', transformed_image)
      #5
      transformed_image = horizontal_flip(transformed_image)
      transformed_image = blur_image(transformed_image)
      aug += 1
      io.imsave(folder_path + 'aug' + str(aug) + '.jpg', transformed_image)
      #6
      transformed_image = gray(image_to_transform)
      transformed_image = warp_shift(transformed_image)
      aug += 1
      io.imsave(folder_path + 'aug' + str(aug) + '.jpg', transformed_image)

In [8]:

from fastai import *
from fastai.vision import *
from fastai.metrics import accuracy, error_rate
defaults.device = torch.device('cuda')

ModuleNotFoundError: No module named 'fastai'

In [9]:
pip install fastai

  Using cached fastai-1.0.61-py3-none-any.whl (239 kB)
  Using cached beautifulsoup4-4.9.1-py3-none-any.whl (115 kB)
  Using cached torch-1.5.1-cp38-none-macosx_10_9_x86_64.whl (80.6 MB)
Processing /Users/amithasanshuvo/Library/Caches/pip/wheels/b9/b1/68/cb4feab29709d4155310d29a421389665dcab9eb3b679b527b/nvidia_ml_py3-7.352.0-py3-none-any.whl
  Using cached torchvision-0.6.1-cp38-cp38-macosx_10_9_x86_64.whl (436 kB)
  Using cached pynvx-1.0.0.tar.gz (150 kB)
  Using cached fastprogress-0.2.3-py3-none-any.whl (12 kB)
  Using cached numexpr-2.7.1-cp38-cp38-macosx_10_9_x86_64.whl (101 kB)
  Using cached soupsieve-2.0.1-py3-none-any.whl (32 kB)
Processing /Users/amithasanshuvo/Library/Caches/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4/future-0.18.2-py3-none-any.whl
  ERROR: Command errored out with exit status 1:
   command: /Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folde

Failed to build pynvx
    Running setup.py install for pynvx ... error
    ERROR: Command errored out with exit status 1:
     command: /Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/z3/3qb81ywd39b6xnfm4sqcqmb40000gn/T/pip-install-o4yr0ycb/pynvx/setup.py'"'"'; __file__='"'"'/private/var/folders/z3/3qb81ywd39b6xnfm4sqcqmb40000gn/T/pip-install-o4yr0ycb/pynvx/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /private/var/folders/z3/3qb81ywd39b6xnfm4sqcqmb40000gn/T/pip-record-j7h8ltnb/install-record.txt --single-version-externally-managed --compile --install-headers /Library/Frameworks/Python.framework/Versions/3.8/include/python3.8/pynvx
         cwd: /private/var/folders/z3/3qb81ywd39b6xnfm4sqcqmb40000gn/T/pip-install-o4yr0ycb/pynvx/
    Complete output 

In [10]:
pip install pynvx

  Using cached pynvx-1.0.0.tar.gz (150 kB)
  ERROR: Command errored out with exit status 1:
   command: /Library/Frameworks/Python.framework/Versions/3.8/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/z3/3qb81ywd39b6xnfm4sqcqmb40000gn/T/pip-install-n7jeavim/pynvx/setup.py'"'"'; __file__='"'"'/private/var/folders/z3/3qb81ywd39b6xnfm4sqcqmb40000gn/T/pip-install-n7jeavim/pynvx/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/z3/3qb81ywd39b6xnfm4sqcqmb40000gn/T/pip-wheel-2wd2j6hs
       cwd: /private/var/folders/z3/3qb81ywd39b6xnfm4sqcqmb40000gn/T/pip-install-n7jeavim/pynvx/
  Complete output (51 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.9-x86_64-3.8
  creating build/lib.macosx-10.9-x86_64-3.8/pynvx
  copying pynvx/

Note: you may need to restart the kernel to use updated packages.
